## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for RxNorm Concept Unique Identifier (RxCUI) Sentence Entity Resolver

- **Model**: `rxcui_resolver_pipeline`
- **Model Description**: This advanced pipeline extracts medical entities from clinical texts and utilizes the `sbiobert_base_cased_mli` Sentence Bert Embeddings to map these entities to their corresponding RxNorm Concept Unique Identifier (RxCUI) codes.

In [ ]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "rxcui-resolver-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [7]:
docs = [
    "She is given folic acid 1 mg daily , levothyroxine 0.1 mg and aspirin 81 mg daily.",
    "The patient was advised to take Omeprazole 20 mg daily before breakfast for acid reflux.",
]



sample_text = """He was seen by the endocrinology service and she was discharged on 50 mg of eltrombopag oral at night, 5 mg amlodipine with meals, and metformin 1000 mg two times a day."""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,0,1,2
0,"{'ner_chunk': '50 mg of eltrombopag oral', 'begin': 67, 'end': 91, 'ner_label': 'DRUG', 'ner_confidence': '0.72838', 'code': '825427', 'resolution': 'eltrombopag 50 MG Oral Tablet', 'all_k_codes': '825427:::2169316:::2049111:::1804806:::1940374:::1597587:::2049117:::1804804:::1597593:::1606862:::310007:::1940376:::2268229:::2256945:::197323:::832080:::213091:::1747574:::1656056:::2361298:::314277:::1424916:::2256947:::805663:::242339', 'all_k_resolutions': 'eltrombopag 50 MG Oral Tablet:::alpelisib 50 MG Oral Tablet:::encorafenib 50 MG Oral Capsule:::alteplase 50 MG Injection [Activase]:::enasidenib 50 MG Oral Tablet:::olaparib 50 MG Oral Capsule:::encorafenib 50 MG Oral Capsule [Braftovi]:::alteplase 50 MG Injection:::olaparib 50 MG Oral Capsule [Lynparza]:::ivacaftor 50 MG Oral Granules:::dolasetron 50 MG Oral Tablet:::enasidenib 50 MG Oral Tablet [Idhifa]:::ubrogepant 50 MG Oral Tablet:::lasmiditan 50 MG Oral Tablet:::altretamine 50 MG Oral Capsule:::pamabrom 50 MG Oral Capsule:::dolasetron 50 MG Oral Tablet [Anzemet]:::venetoclax 50 MG Oral Tablet:::cangrelor 50 MG Injection:::tucatinib 50 MG Oral Tablet:::venlafaxine 50 MG Oral Tablet:::dabrafenib 50 MG Oral Capsule:::lasmiditan 50 MG Oral Tablet [Reyvow]:::armodafinil 50 MG Oral Tablet [Nuvigil]:::prasterone 50 MG Oral Capsule', 'all_k_distances': '4.3863:::6.1202:::6.1623:::6.2046:::6.3518:::6.4410:::6.4845:::6.5186:::6.5723:::6.5798:::6.7074:::6.7092:::6.7096:::6.7196:::6.7407:::6.7514:::6.7546:::6.7600:::6.7941:::6.8422:::6.8493:::6.8817:::6.9487:::6.9651:::6.9651'}","{'ner_chunk': '5 mg amlodipine', 'begin': 103, 'end': 117, 'ner_label': 'DRUG', 'ner_confidence': '0.5661333', 'code': '197361', 'resolution': 'amlodipine 5 MG Oral Tablet', 'all_k_codes': '197361:::2377371:::387013:::212549:::311354:::2377373:::402131:::310672:::308136:::198286:::351249:::251857:::404602:::198105:::311386:::403957:::1998462:::855818:::204416:::198189:::1649485:::859981:::1364498:::2377367:::1928948', 'all_k_resolutions': 'amlodipine 5 MG Oral Tablet:::levamlodipine 5 MG Oral Tablet:::nebivolol 5 MG Oral Tablet:::amlodipine 5 MG Oral Tablet [Norvasc]:::lisinopril 5 MG Oral Tablet:::levamlodipine 5 MG Oral Tablet [Conjupri]:::aripiprazole 5 MG Oral Tablet:::haloperidol 5 MG Oral Tablet:::amlodipine 2.5 MG Oral Tablet:::timolol 5 MG Oral Tablet:::escitalopram 5 MG Oral Tablet:::ramipril 5 MG Oral Tablet:::aripiprazole 5 MG Oral Tablet [Abilify]:::pindolol 5 MG Oral Tablet:::loxapine 5 MG Oral Capsule:::tadalafil 5 MG Oral Tablet:::Sensor aripiprazole 5 MG Oral Tablet:::prasugrel 5 MG Oral Tablet:::haloperidol 5 MG/ML Injectable Solution:::ramipril 5 MG Oral Capsule:::ivabradine 5 MG Oral Tablet:::asenapine 5 MG Sublingual Tablet:::lomitapide 5 MG Oral Capsule [Juxtapid]:::levamlodipine 2.5 MG Oral Tablet:::dronabinol 5 MG/ML Oral Solution', 'all_k_distances': '5.8704:::6.9994:::8.1125:::8.1422:::8.1812:::8.3356:::8.5179:::8.6665:::8.7202:::8.7215:::8.7282:::8.8417:::8.8832:::8.9053:::8.9880:::8.9968:::9.0577:::9.0718:::9.1269:::9.1561:::9.2993:::9.3102:::9.3740:::9.3744:::9.3887'}","{'ner_chunk': 'metformin 1000 mg', 'begin': 135, 'end': 151, 'ner_label': 'DRUG', 'ner_confidence': '0.6566333', 'code': '861004', 'resolution': 'metformin hydrochloride 1000 MG Oral Tablet', 'all_k_codes': '861004:::1665093:::1791593:::311625:::1665050:::1722919:::847074:::1665102:::1721458:::1734683:::1721475:::2265706:::1664981:::1791595:::636337:::312576:::314234:::861006:::1656313:::239180:::1659283:::1719003:::1734686:::1722939:::259239', 'all_k_resolutions': 'metformin hydrochloride 1000 MG Oral Tablet:::cefepime 1000 MG Injection:::ifosfamide 1000 MG Injection:::methotrexate 1000 MG Injection:::cefazolin 1000 MG Injection:::cefotetan 1000 MG Injection:::taurine 1000 MG Oral Capsule:::cefoxitin 1000 MG Injection:::nafcillin 1000 MG Injection:::ertapenem 1000 MG Injection:::ampicillin 1000 MG Injection:::cefiderocol 1000 MG Injection:::aztreonam 1000 MG Injection:::ifosfamide 1000 MG Injection [

#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [9]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,0,1,2
0,"{'ner_chunk': 'folic acid 1 mg', 'begin': 13, 'end': 27, 'ner_label': 'DRUG', 'ner_confidence': '0.67852503', 'code': '310410', 'resolution': 'folic acid 1 MG Oral Tablet', 'all_k_codes': '310410:::846934:::245557:::253083:::1601112:::242698:::476628:::1423423:::1539820:::309594:::199364:::310313:::309593:::747193:::2168763:::1718589:::808611:::309595:::1537458:::198640:::197658:::847004:::198641:::351121:::542045', 'all_k_resolutions': 'folic acid 1 MG Oral Tablet:::mecobalamin 1 MG Oral Capsule:::vitamin B12 1 MG Oral Capsule:::folic acid 0.8 MG / vitamin B12 1 MG Oral Tablet:::mecobalamin 1 MG Oral Lozenge:::vitamin B12 1 MG Oral Lozenge:::folic acid 1 MG / vitamin B12 0.5 MG Oral Tablet:::vitamin B12 1 MG/ML Oral Solution:::mecobalamin 1 MG Sublingual Tablet:::vitamin B12 1 MG/ML Injectable Solution:::vitamin B12 0.1 MG Oral Lozenge:::ferrous fumarate 324 MG / folic acid 1 MG Oral Tablet:::vitamin B12 1 MG Oral Tablet:::topotecan 1 MG Oral Capsule:::tesamorelin 1 MG Injection:::trabectedin 1 MG Injection:::folic acid 0.4 MG / vitamin B12 1 MG Sublingual Tablet:::vitamin B12 0.1 MG Oral Tablet:::vitamin K2 0.1 MG Oral Capsule:::folic acid 0.4 MG Oral Tablet:::estradiol 1 MG Oral Tablet:::vitamin K 0.1 MG Oral Capsule:::folic acid 0.8 MG Oral Tablet:::minocycline 1 MG Oral Powder:::folic acid 1 MG / vitamin B12 0.5 MG Oral Tablet [Foltrate]', 'all_k_distances': '5.7442:::9.2313:::9.5890:::9.8275:::10.0312:::10.0557:::10.0979:::10.1475:::10.2895:::10.3375:::10.3431:::10.3516:::10.3565:::10.3957:::10.4176:::10.4224:::10.4279:::10.4507:::10.4579:::10.5116:::10.5552:::10.6201:::10.6259:::10.7205:::10.7346'}","{'ner_chunk': 'levothyroxine 0.1 mg', 'begin': 37, 'end': 56, 'ner_label': 'DRUG', 'ner_confidence': '0.8457667', 'code': '1115267', 'resolution': 'levothyroxine sodium 0.1 MG Injection', 'all_k_codes': '1115267:::966404:::892246:::2056480:::966412:::966178:::966177:::2056492:::966283:::966233:::966219:::1115269:::2056482:::892251:::966415:::966250:::1923050:::966406:::2056494:::966220:::903305:::966221:::2056462:::966434:::2056508', 'all_k_resolutions': 'levothyroxine sodium 0.1 MG Injection:::levothyroxine sodium 0.1 MG Oral Capsule:::levothyroxine sodium 0.1 MG Oral Tablet:::levothyroxine sodium 0.1 MG/ML Oral Solution:::levothyroxine sodium 0.013 MG Oral Capsule:::levothyroxine sodium 0.1 MG Oral Tablet [Levo-T]:::levothyroxine sodium 0.1 MG Oral Tablet [Euthyrox]:::levothyroxine sodium 0.013 MG/ML Oral Solution:::levothyroxine sodium 0.1 MG Oral Tablet [Levoxyl]:::levothyroxine sodium 0.1 MG Oral Tablet [Unithroid]:::levothyroxine sodium 0.5 MG Injection:::levothyroxine sodium 0.2 MG Injection:::levothyroxine sodium 0.1 MG/ML Oral Solution [Tirosint]:::levothyroxine sodium 0.2 MG Oral Tablet:::levothyroxine sodium 0.1 MG Oral Capsule [Tirosint]:::levothyroxine sodium 0.1 MG Oral Tablet [Synthroid]:::levothyroxine sodium 0.2 MG Oral Capsule:::levothyroxine sodium 0.025 MG Oral Capsule:::levothyroxine sodium 0.013 MG/ML Oral Solution [Tirosint]:::levothyroxine sodium 0.025 MG Oral Tablet:::levothyroxine sodium 0.1 MG / liothyronine sodium 0.025 MG Oral Tablet:::levothyroxine sodium 0.05 MG Oral Tablet:::levothyroxine sodium 0.025 MG/ML Oral Solution:::levothyroxine sodium 0.013 MG Oral Capsule [Tirosint]:::levothyroxine sodium 0.2 MG/ML Oral Solution', 'all_k_distances': '4.4075:::5.2968:::5.9096:::6.1731:::6.4272:::6.8615:::6.8961:::6.9876:::7.1843:::7.2156:::7.4194:::7.6961:::7.7001:::7.8607:::7.8886:::7.9190:::7.9832:::8.0875:::8.0901:::8.0907:::8.1956:::8.4002:::8.4002:::8.4422:::8.4554'}","{'ner_chunk': 'aspirin 81 mg', 'begin': 62, 'end': 74, 'ner_label': 'DRUG', 'ner_confidence': '0.72886664', 'code': '243670', 'resolution': 'aspirin 81 MG Oral Tablet', 'all_k_codes': '243670:::318272:::825180:::308416:::211832:::794229:::1600991:::1189781:::1293665:::749795:::198475:::1052678:::1722689:::212033:::211874:::830530:::1537811:::260847:::211881:::200094:::309362:::1811631:::1537813:::308411:::1868018', 'all_

### JSON Lines

In [10]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [11]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "folic acid 1 mg", "begin": 13, "end": 27, "ner_label": "DRUG", "ner_confidence": "0.67852503", "code": "310410", "resolution": "folic acid 1 MG Oral Tablet", "all_k_codes": "310410:::846934:::245557:::253083:::1601112:::242698:::476628:::1423423:::1539820:::309594:::199364:::310313:::309593:::747193:::2168763:::1718589:::808611:::309595:::1537458:::198640:::197658:::847004:::198641:::351121:::542045", "all_k_resolutions": "folic acid 1 MG Oral Tablet:::mecobalamin 1 MG Oral Capsule:::vitamin B12 1 MG Oral Capsule:::folic acid 0.8 MG / vitamin B12 1 MG Oral Tablet:::mecobalamin 1 MG Oral Lozenge:::vitamin B12 1 MG Oral Lozenge:::folic acid 1 MG / vitamin B12 0.5 MG Oral Tablet:::vitamin B12 1 MG/ML Oral Solution:::mecobalamin 1 MG Sublingual Tablet:::vitamin B12 1 MG/ML Injectable Solution:::vitamin B12 0.1 MG Oral Lozenge:::ferrous fumarate 324 MG / folic acid 1 MG Oral Tablet:::vitamin B12 1 MG Oral Tablet:::topotecan 1 MG Oral Capsule:::tesamorelin 1 M

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [15]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

INFO:sagemaker:Creating transform job with name: rxcui-resolver-pipeline-en-2024-12-03-08-06-02-746


.................................................24/12/03 08:14:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
#015[Stage 0:>                                                          (0 + 1) / 1]#015#015                                                                                #015INFO:     Started server process [7]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
📋 Loading license number 0 from /root/.johnsnowlabs/licenses/license_number_{number}_for_Spark-Healthcare_Spark-OCR.json
👌 Launched #033cpu optimized#033 session with with: 🚀Spark-NLP==5.5.0, 💊Spark-Healthcare==5.5.0, running on ⚡ PySpark==3.4.0
INFO:     169.254.255.130:44690 - "GET /ping HTTP/1.1" 200 OK
IN

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

,0,1,2
0,"{'ner_chunk': 'folic acid 1 mg', 'begin': 13, 'end': 27, 'ner_label': 'DRUG', 'ner_confidence': '0.67852503', 'code': '310410', 'resolution': 'folic acid 1 MG Oral Tablet', 'all_k_codes': '310410:::846934:::245557:::253083:::1601112:::242698:::476628:::1423423:::1539820:::309594:::199364:::310313:::309593:::747193:::2168763:::1718589:::808611:::309595:::1537458:::198640:::197658:::847004:::198641:::351121:::542045', 'all_k_resolutions': 'folic acid 1 MG Oral Tablet:::mecobalamin 1 MG Oral Capsule:::vitamin B12 1 MG Oral Capsule:::folic acid 0.8 MG / vitamin B12 1 MG Oral Tablet:::mecobalamin 1 MG Oral Lozenge:::vitamin B12 1 MG Oral Lozenge:::folic acid 1 MG / vitamin B12 0.5 MG Oral Tablet:::vitamin B12 1 MG/ML Oral Solution:::mecobalamin 1 MG Sublingual Tablet:::vitamin B12 1 MG/ML Injectable Solution:::vitamin B12 0.1 MG Oral Lozenge:::ferrous fumarate 324 MG / folic acid 1 MG Oral Tablet:::vitamin B12 1 MG Oral Tablet:::topotecan 1 MG Oral Capsule:::tesamorelin 1 MG Injection:::trabectedin 1 MG Injection:::folic acid 0.4 MG / vitamin B12 1 MG Sublingual Tablet:::vitamin B12 0.1 MG Oral Tablet:::vitamin K2 0.1 MG Oral Capsule:::folic acid 0.4 MG Oral Tablet:::estradiol 1 MG Oral Tablet:::vitamin K 0.1 MG Oral Capsule:::folic acid 0.8 MG Oral Tablet:::minocycline 1 MG Oral Powder:::folic acid 1 MG / vitamin B12 0.5 MG Oral Tablet [Foltrate]', 'all_k_distances': '5.7442:::9.2313:::9.5890:::9.8275:::10.0312:::10.0557:::10.0979:::10.1475:::10.2895:::10.3375:::10.3431:::10.3516:::10.3565:::10.3957:::10.4176:::10.4224:::10.4279:::10.4507:::10.4579:::10.5116:::10.5552:::10.6201:::10.6259:::10.7205:::10.7346'}","{'ner_chunk': 'levothyroxine 0.1 mg', 'begin': 37, 'end': 56, 'ner_label': 'DRUG', 'ner_confidence': '0.8457667', 'code': '1115267', 'resolution': 'levothyroxine sodium 0.1 MG Injection', 'all_k_codes': '1115267:::966404:::892246:::2056480:::966412:::966178:::966177:::2056492:::966283:::966233:::966219:::1115269:::2056482:::892251:::966415:::966250:::1923050:::966406:::2056494:::966220:::903305:::966221:::2056462:::966434:::2056508', 'all_k_resolutions': 'levothyroxine sodium 0.1 MG Injection:::levothyroxine sodium 0.1 MG Oral Capsule:::levothyroxine sodium 0.1 MG Oral Tablet:::levothyroxine sodium 0.1 MG/ML Oral Solution:::levothyroxine sodium 0.013 MG Oral Capsule:::levothyroxine sodium 0.1 MG Oral Tablet [Levo-T]:::levothyroxine sodium 0.1 MG Oral Tablet [Euthyrox]:::levothyroxine sodium 0.013 MG/ML Oral Solution:::levothyroxine sodium 0.1 MG Oral Tablet [Levoxyl]:::levothyroxine sodium 0.1 MG Oral Tablet [Unithroid]:::levothyroxine sodium 0.5 MG Injection:::levothyroxine sodium 0.2 MG Injection:::levothyroxine sodium 0.1 MG/ML Oral Solution [Tirosint]:::levothyroxine sodium 0.2 MG Oral Tablet:::levothyroxine sodium 0.1 MG Oral Capsule [Tirosint]:::levothyroxine sodium 0.1 MG Oral Tablet [Synthroid]:::levothyroxine sodium 0.2 MG Oral Capsule:::levothyroxine sodium 0.025 MG Oral Capsule:::levothyroxine sodium 0.013 MG/ML Oral Solution [Tirosint]:::levothyroxine sodium 0.025 MG Oral Tablet:::levothyroxine sodium 0.1 MG / liothyronine sodium 0.025 MG Oral Tablet:::levothyroxine sodium 0.05 MG Oral Tablet:::levothyroxine sodium 0.025 MG/ML Oral Solution:::levothyroxine sodium 0.013 MG Oral Capsule [Tirosint]:::levothyroxine sodium 0.2 MG/ML Oral Solution', 'all_k_distances': '4.4075:::5.2968:::5.9096:::6.1731:::6.4272:::6.8615:::6.8961:::6.9876:::7.1843:::7.2156:::7.4194:::7.6961:::7.7001:::7.8607:::7.8886:::7.9190:::7.9832:::8.0875:::8.0901:::8.0907:::8.1956:::8.4002:::8.4002:::8.4422:::8.4554'}","{'ner_chunk': 'aspirin 81 mg', 'begin': 62, 'end': 74, 'ner_label': 'DRUG', 'ner_confidence': '0.72886664', 'code': '243670', 'resolution': 'aspirin 81 MG Oral Tablet', 'all_k_codes': '243670:::318272:::825180:::308416:::211832:::794229:::1600991:::1189781:::1293665:::749795:::198475:::1052678:::1722689:::212033:::211874:::830530:::1537811:::260847:::211881:::200094:::309362:::1811631:::1537813:::308411:::1868018', 'all_

### JSON Lines

In [18]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

INFO:sagemaker:Creating model with name: rxcui-resolver-pipeline-en-2024-12-03-08-15-38-722
INFO:sagemaker:Creating transform job with name: rxcui-resolver-pipeline-en-2024-12-03-08-15-43-773


.........................................24/12/03 08:22:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-12-03T08:23:29.748:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
#015[Stage 0:>                                                          (0 + 1) / 1]#015#015                                                                                #015INFO:     Started server process [7]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
📋 Loading license number 0 from /root/.johnsnowlabs/licenses/license_number_{number}_for_Spark-Healthcare_Spark-OCR.json
👌 Launched #033cpu optimized#033 session with with: 🚀Spark-NLP==5.5.0, 💊Spark-Healt

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "folic acid 1 mg", "begin": 13, "end": 27, "ner_label": "DRUG", "ner_confidence": "0.67852503", "code": "310410", "resolution": "folic acid 1 MG Oral Tablet", "all_k_codes": "310410:::846934:::245557:::253083:::1601112:::242698:::476628:::1423423:::1539820:::309594:::199364:::310313:::309593:::747193:::2168763:::1718589:::808611:::309595:::1537458:::198640:::197658:::847004:::198641:::351121:::542045", "all_k_resolutions": "folic acid 1 MG Oral Tablet:::mecobalamin 1 MG Oral Capsule:::vitamin B12 1 MG Oral Capsule:::folic acid 0.8 MG / vitamin B12 1 MG Oral Tablet:::mecobalamin 1 MG Oral Lozenge:::vitamin B12 1 MG Oral Lozenge:::folic acid 1 MG / vitamin B12 0.5 MG Oral Tablet:::vitamin B12 1 MG/ML Oral Solution:::mecobalamin 1 MG Sublingual Tablet:::vitamin B12 1 MG/ML Injectable Solution:::vitamin B12 0.1 MG Oral Lozenge:::ferrous fumarate 324 MG / folic acid 1 MG Oral Tablet:::vitamin B12 1 MG Oral Tablet:::topotecan 1 MG Oral Capsule:::tesamorelin 1 M

In [21]:
model.delete_model()

INFO:sagemaker:Deleting model with name: rxcui-resolver-pipeline-en-2024-12-03-08-15-38-722


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

